# Akbank Derin Öğrenme Proje Kampı: MR Görüntülerinde Beyin Tümörü Sınıflandırması

## Projenin Amacı
Bu proje kapsamında, Manyetik Rezonans (MR) görüntülerini kullanarak dört farklı beyin durumu (Glioma, Meningioma, Pituitary tümörleri ve No Tumor - Tümör Yok) arasında sınıflandırma yapabilen güçlü bir **Derin Öğrenme modeli** geliştirmek amaçlanmıştır. Projenin temel hedefi, yüksek doğrulukla sınıflandırma yaparak derin öğrenme modelinin tıbbi görüntüleme alanındaki etkinliğini göstermektir.

## Veri Seti Hakkında Bilgi
* **Adı:** Brain Tumor MRI Dataset
* **Tür:** Multiclass Image Classification (Çoklu Sınıf Görüntü Sınıflandırması)
* **Sınıf Sayısı:** 4
* **Sınıflar:** Glioma, Meningioma, Pituitary ve No Tumor
* **Boyut:** Toplam 7.022 görüntü
* **Denge:** Sınıflar arasındaki görüntü sayısı dağılımı oldukça dengelidir.

## Kullanılan Temel Yöntemler (Güncellenmiş)

* **Geliştirme Ortamı:** Kaggle Notebook / Google Colab
* **Ön İşleme:** Görüntülerin **(224, 224) piksel** boyutuna yeniden boyutlandırılması ve **0 ile 1** arasına normalizasyonu.
* **Veri Çoğaltma (Data Augmentation):** Modelin aşırı öğrenmesini engellemek amacıyla rotasyon, çevirme ve kaydırma gibi dönüşümlerin uygulanması.
* **Model Mimarisi:** Mentör geri bildirimleri doğrultusunda, aşırı öğrenme eğilimini yapısal olarak çözmek için **VGG16 Temel Modeli** (Transfer Öğrenme) kullanılmıştır. Modelin Evrişim katmanları dondurulmuş, üzerine **Dropout (0.5)** ve **Dense katmanları** eklenmiştir.
* **Hiperparametre Optimizasyonu:** Aşırı öğrenme problemine çözüm olarak **Transfer Öğrenme** stratejisi uygulanmıştır. Modelin genelleme yeteneğini artırmak amacıyla düşük bir öğrenme oranı (`0.0001`) kullanılmış; ilerideki iyileştirmeler için ise **L2 Regularizasyon** ve **Fine-Tuning** önerileri belirlenmiştir.

## 1. Geliştirme Ortamı ve Temel Kütüphaneler

Bu başlangıç hücreleri, Kaggle Notebook ortamının standardını oluşturur.

* **Amaç:** Veri analizi için temel kütüphaneleri (NumPy, Pandas, OS) yüklemek ve Kaggle'da bulunan veri setlerinin (Brain Tumor MRI Dataset) dosya yollarını doğrulamaktır.
* Bu adım, projenin **Geliştirme Ortamı (Adım 1)** gereksinimlerinin bir parçası olarak, veri setine doğru erişim sağlandığını teyit eder.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        # print(os.path.join(dirname, filename)) <--- BU SATIRI SİLİN VEYA BAŞINA # KOYUN
        pass # Artık hiçbir şey yazdırmayacak

# Tüm temel kütüphaneler yüklenmiştir.

## 2.Veri Keşfi ve Görselleştirme

Bu adımda, eğitim (Training) klasöründeki alt klasörler incelenmiş ve her sınıfta bulunan görüntü sayıları listelenmiştir. 
Ayrıca, her sınıftan rastgele bir görüntü görselleştirilmiştir. 

- **Glioma, Meningioma, Pituitary, No Tumor** olmak üzere toplam 4 sınıf vardır.  
- Sınıflar arasındaki örnek sayıları yaklaşık olarak dengelidir (1300-1600 arası).  
- Görseller MRI (Manyetik Rezonans) görüntüleridir ve farklı açılardan alınmıştır.  

Bu adım, veri setinin yapısını anlamak ve sonraki aşamada uygulanacak ön işleme tekniklerine zemin hazırlamak için yapılmıştır.

In [ ]:
import matplotlib.pyplot as plt
import cv2
import random
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
import numpy as np

# -----------------------------------------------------------
# KRİTİK HİPERPARAMETRELER
# -----------------------------------------------------------
IMAGE_SIZE = (224, 224) 
BATCH_SIZE = 32         
EPOCHS = 30             
NUM_CLASSES = 4         
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# -----------------------------------------------------------
# VERİ YOLU TANIMLARI 
# -----------------------------------------------------------
data_dir = '/kaggle/input/brain-tumor-mri-dataset' 
train_path = os.path.join(data_dir, 'Training') # Bu satır train_path'i tanımlar
test_path = os.path.join(data_dir, 'Testing')

print(f"train_path değişkeni başarıyla tanımlandı: {train_path}")

print("\n--- Eğitim Klasöründeki Sınıf Dağılımı ---")
# Her sınıftaki görüntü sayısını sayma
class_counts = {}
for folder in os.listdir(train_path):
    folder_path = os.path.join(train_path, folder)
    if os.path.isdir(folder_path):
        count = len(os.listdir(folder_path))
        class_counts[folder] = count
        print(f"{folder}: {count} görüntü")

# -------------------------------------------------------------------
# Her sınıftan rastgele bir örneği görselleştirme
# -------------------------------------------------------------------
class_names = list(class_counts.keys())
num_classes = len(class_names)

if num_classes > 0:
    fig, axs = plt.subplots(1, num_classes, figsize=(15, 5))
    
    # 4 sınıflı bir görselleştirme için döngü
    for i, folder in enumerate(class_names):
        try:
            folder_path = os.path.join(train_path, folder)
            # Klasör içinden rastgele bir görüntü seç
            img_name = random.choice(os.listdir(folder_path))
            img_path = os.path.join(folder_path, img_name)
            
            # OpenCV ile görüntüyü oku (BGR formatında gelir)
            img = cv2.imread(img_path)
            
            # Matplotlib için BGR'den RGB'ye dönüştür
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            axs[i].imshow(img)
            axs[i].set_title(f"{folder}\n({class_counts[folder]} adet)")
            axs[i].axis("off")
            
        except Exception as e:
            # Hata oluşursa (örneğin boş klasör) atla
            print(f"Hata oluştu: {folder} klasörü görselleştirilemedi. Hata: {e}")
            
    plt.tight_layout()
    plt.show()
else:
    print("Hata: Eğitim klasöründe alt sınıf klasörü bulunamadı.")

## 2.1 Veri Önişleme, Normalizasyon ve Veri Çoğaltma (Data Augmentation)

Modelin aşırı öğrenmesini (overfitting) engellemek ve genelleme yeteneğini artırmak amacıyla **ImageDataGenerator** kullanılmıştır.

1.  **Normalizasyon:** Tüm piksel değerleri $0-255$ aralığından $0-1$ aralığına ölçeklenmiştir (`rescale=1./255`).
2.  **Veri Çoğaltma:** Eğitim veri setine özgü olarak aşağıdaki dönüşümler uygulanmıştır:
    * `rotation_range=20`: Görüntüler $\pm 20$ derece rastgele döndürülmüştür.
    * `horizontal_flip=True`: Yatay çevirme uygulanmıştır.
    * `zoom_range=0.1`: %10'a kadar rastgele yakınlaştırma yapılmıştır.
    * `width_shift_range` ve `height_shift_range`: Görüntü %10'a kadar yatay ve dikey olarak kaydırılmıştır.

Doğrulama (Validation) ve Test setlerine yalnızca normalizasyon uygulanmış, çoğaltma yapılmamıştır. Bu sayede modelin gerçek verilere karşı performansı doğru bir şekilde ölçülecektir. Veri seti %80 Eğitim, %20 Doğrulama olarak ayrılmıştır.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# -----------------------------------------------------------
# 1. PARAMETRELER
# -----------------------------------------------------------
IMAGE_SIZE = (224,224 ) # Görüntülerin yeniden boyutlandırılacağı boyut 
BATCH_SIZE = 32         # Bir adımda işlenecek görüntü sayısı
SEED = 42               # Rastgelelik için sabit tohum

data_dir = '/kaggle/input/brain-tumor-mri-dataset' 
train_path = os.path.join(data_dir, 'Training')
test_path = os.path.join(data_dir, 'Testing')

# -----------------------------------------------------------
# 2. VERİ ÇOĞALTMA VE NORMALİZASYON TANIMLAMALARI
# -----------------------------------------------------------

# Eğitim Verisi için Data Augmentation (Çoğaltma)
train_datagen = ImageDataGenerator(
    rescale=1./255,                 # Normalizasyon (0-1 arasına ölçekleme)
    rotation_range=20,              # 20 dereceye kadar rastgele döndürme
    width_shift_range=0.1,          # %10'a kadar yatay kaydırma
    height_shift_range=0.1,         # %10'a kadar dikey kaydırma
    shear_range=0.1,                # Rastgele kesme
    zoom_range=0.1,                 # %10'a kadar rastgele yakınlaştırma
    horizontal_flip=True,           # Yatay çevirme
    fill_mode='nearest',            # Boş kalan pikselleri doldurma metodu
    validation_split=0.2            # Eğitim verisinin %20'sini doğrulama (validation) için ayır
)

# Test Verisi için YALNIZCA Normalizasyon
test_datagen = ImageDataGenerator(
    rescale=1./255                  # Normalizasyon
)

In [ ]:
# -----------------------------------------------------------
# 3. VERİ AKIŞLARININ OLUŞTURULMASI (Data Flow)
# -----------------------------------------------------------

# Eğitim Seti (Augmentation uygulanmış)
train_generator = train_datagen.flow_from_directory(
    train_path,                     # Eğitim veri klasörü
    target_size=IMAGE_SIZE,         # Tüm görüntüleri bu boyuta getir
    batch_size=BATCH_SIZE,
    class_mode='categorical',       # Çoklu sınıflandırma (4 sınıf)
    subset='training',              # Tanımlanan split'in eğitim kısmını al
    seed=SEED
)

# Doğrulama Seti (Sadece normalizasyon)
validation_generator = train_datagen.flow_from_directory(
    train_path,                     # Aynı eğitim veri klasörünü kullan
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',            # Tanımlanan split'in doğrulama kısmını al
    seed=SEED
)

# Test Seti (Sadece normalizasyon)
test_generator = test_datagen.flow_from_directory(
    test_path,                      # Test veri klasörü
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,                
    seed=SEED
)

# Sınıf İsimlerini Kontrol Etme
class_names = list(train_generator.class_indices.keys())
print("\nSınıf İsimleri (Class Names):", class_names)
print("Toplam Eğitim Örneği:", train_generator.samples)
print("Toplam Doğrulama Örneği:", validation_generator.samples)
print("Toplam Test Örneği:", test_generator.samples)

# 3. CNN Model Mimarisi ve Eğitimi

Projenin temel hedefi olan genelleme yeteneğini artırmak ve önceki denemelerde gözlemlenen şiddetli aşırı öğrenmeyi çözmek amacıyla **VGG16 Transfer Öğrenme** mimarisine geçilmiştir.

**Model Mimarisi:**
1.  **Temel Model (Base Model):** VGG16 mimarisi, ImageNet veri kümesi üzerinde eğitilmiş ağırlıklarla (`weights='imagenet'`) yüklenmiştir.
2.  **Katman Dondurma (Freezing):** VGG16'nın tüm Evrişim (Convolutional) katmanları dondurulmuştur (`layer.trainable = False`). Bu, modelin genel görüntü özelliklerini korurken, eğitilebilir parametre sayısını önemli ölçüde azaltarak aşırı öğrenme riskini düşürmüştür.
3.  **Sınıflandırma Başlığı (Head):** Dondurulmuş katmanların çıktısının üzerine, Flatten, Dense ve **%50 Dropout** katmanlarından oluşan yeni bir sınıflandırma başlığı eklenmiştir.
4.  **Çıktı Katmanı:** Model, 4 sınıflı çıktı üretmek üzere Softmax aktivasyonu ile sonlandırılmıştır.

**Derleme ve Eğitim Parametreleri:**
* **Optimizer:** Adam
* **Learning Rate:** Transfer Öğrenme için ideal olan daha düşük bir öğrenme oranı (`0.0001`) kullanılmıştır.
* **Loss Function:** Categorical Cross-entropy (Çoklu sınıflandırma)
* **Metrics:** Accuracy

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

LEARNING_RATE = 0.0001
EPOCHS = 10 

# -----------------------------------------------------------
# 3.1. TRANSFER ÖĞRENME MODELİNİ (VGG16) OLUŞTURMA (Ağırlıksız)
# -----------------------------------------------------------

input_shape = (IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
base_model = VGG16(weights=None,  
                   include_top=False, 
                   input_shape=input_shape)

# Modelin üzerine kendi sınıflandırma başlığımızı (Head) ekliyoruz
x = base_model.output
x = Flatten()(x)                
x = Dense(512, activation='relu')(x) 
x = Dropout(0.5)(x)             
predictions = Dense(NUM_CLASSES, activation='softmax')(x) 

# Son modeli tanımlama
model_vgg16 = Model(inputs=base_model.input, outputs=predictions)

# -----------------------------------------------------------
# 3.2. Modeli Derleme (Compile)
# -----------------------------------------------------------

model_vgg16.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE), 
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Model Özetini Görüntüleme
model_vgg16.summary()

# -----------------------------------------------------------
# 3.3. Modelin Eğitilmesi (Fit) ve KAYDEDİLMESİ
# -----------------------------------------------------------

checkpoint = ModelCheckpoint(
    filepath='vgg16_en_iyi_model.h5', # Dosya bu isimle kaydedilecek
    monitor='val_loss',               # Doğrulama hatasını takip et
    save_best_only=True,              # Sadece en iyi sonucu vereni kaydet (Kötüleşirse kaydetme)
    mode='min',                       # Loss'un düşmesi iyi olduğu için 'min' seçiyoruz
    verbose=1                         # Kaydettiğinde ekrana yazı yazsın
)

history = model_vgg16.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# 4. Modelin Değerlendirilmesi ve Analizi

Aşırı öğrenme sorununun çözümü amacıyla **VGG16 Transfer Öğrenme** modeline geçildikten sonra, eğitim metrikleri tekrar incelenmiştir. Yapısal değişikliğin etkisiyle, Eğitim ve Doğrulama metrikleri arasında önceki denemeye göre çok daha sağlıklı bir denge gözlemlenmiştir.

Bu aşamada modelin performansını detaylı incelemek için:
* Accuracy ve Loss Grafikleri
* Confusion Matrix ve Classification Report
* Grad-CAM Görselleştirmesi (Açıklanabilirlik) oluşturulacaktır.

### 4.1 Eğitim Metrikleri Görselleştirilmesi

VGG16 Transfer Öğrenme modelinin **10 epoch'luk** performansı incelenmiştir.

**Gözlemler:**
* **Aşırı Öğrenmede İyileşme:** VGG16'ya geçişin etkisiyle, Eğitim Doğruluğu (%90+) ile Doğrulama Doğruluğu arasındaki fark (eski %20'lik fark yerine) **önemli ölçüde azalmıştır**.
* **Stabilite:** Doğrulama Kaybı (Validation Loss), önceki modeldeki gibi yükselen bir trend yerine, daha stabil ve düşüş eğilimi göstermiştir.
* **Başarı:** Model, çok daha az epoch'ta daha yüksek ve daha dengeli bir doğrulama doğruluğuna ulaşmıştır.

In [ ]:
import matplotlib.pyplot as plt

# Eğitim ve Doğrulama Doğruluğu Grafiği
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu (Training Acc)')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu (Validation Acc)')
plt.title('Doğruluk Eğrileri (Accuracy)')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk (Accuracy)')
plt.legend()
plt.grid(True)

# Eğitim ve Doğrulama Kaybı Grafiği
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Eğitim Kaybı (Training Loss)')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı (Validation Loss)')
plt.title('Kayıp Eğrileri (Loss)')
plt.xlabel('Epoch')
plt.ylabel('Kayıp (Loss)')
plt.legend()
plt.grid(True)

plt.show() 

### 4.2 Eğitim Metrikleri Analizi (Accuracy ve Loss Eğrileri)

Modelin eğitim süreci boyunca elde edilen Doğruluk (Accuracy) ve Kayıp (Loss) grafikleri, VGG16 Transfer Öğrenme mimarisinin başarısını ortaya koymaktadır.

**Grafiklerin Yorumlanması:**

* **Doğruluk Eğrisi (Sol Grafik):** Mavi (Eğitim) ve Turuncu (Doğrulama) çizgileri birbirine paralel bir şekilde yükselme eğilimindedir. Özellikle son epoch'larda Validation Accuracy'nin **%75 seviyelerine** ulaşması, modelin genelleme yeteneğinin arttığını göstermektedir.
* **Kayıp Eğrisi (Sağ Grafik):** Normalde aşırı öğrenme (overfitting) durumunda Turuncu çizginin (Validation Loss) yukarı doğru fırlaması beklenir. Ancak grafiğimizde **turuncu çizginin epoch ilerledikçe aşağı yönlü (azalan) bir trend izlediği** görülmektedir.
* **Genel Başarı:** Loss değerinin düşmesi ve Accuracy'nin artması, modelin sadece eğitim verisini ezberlemediğini, **gerçekten öğrenerek** yeni veriler üzerinde de başarılı tahminler yapabildiğini kanıtlamaktadır.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import seaborn as sns

# 1. Test seti üzerindeki tahminleri al
test_steps = test_generator.samples // test_generator.batch_size + 1
Y_pred = model_vgg16.predict(test_generator, steps=test_steps)

# 2. Tahmin edilen sınıflar (En yüksek olasılıklı sınıfın indeksi)
y_pred = np.argmax(Y_pred, axis=1)

# 3. Gerçek sınıflar (Test generator'dan alınır)
y_true = test_generator.classes

# Sınıf isimlerini al
class_names = list(test_generator.class_indices.keys())

print("--- Sınıflandırma Raporu ---")
print(classification_report(y_true, y_pred, target_names=class_names))

print("\n--- Karışıklık Matrisi (Confusion Matrix) ---")
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Tahmin Edilen Sınıf')
plt.ylabel('Gerçek Sınıf')
plt.title('Karışıklık Matrisi')
plt.show()

### 4.3 Test Seti Sonuçları: Classification Report ve Confusion Matrix Analizi

Yapısal iyileştirme (VGG16 Transfer Öğrenme) sonrasında model, Test Seti üzerinde **%72 Genel Doğruluk (Accuracy)** elde etmiştir. Sınıflandırma raporu ve Karmaşıklık Matrisi (Confusion Matrix) incelendiğinde, modelin bazı sınıflarda mükemmel çalışırken, bazı sınıflarda kararsız kaldığı görülmektedir.

**Kritik Gözlemler ve Analiz:**

* **En Güçlü Yön (Sağlıklı Beyin Tespiti):** Modelin en başarılı olduğu alan **Notumor (Tümör Yok)** sınıfıdır. **%99 Recall** değeri ile model, sağlıklı görüntüleri neredeyse hiç kaçırmadan doğru tespit etmektedir. Bu, tıbbi açıdan "yanlış alarm vermemek" adına çok değerli bir sonuçtur.
* **Başarılı Tümör Tespiti:** **Pituitary (Hipofiz)** tümörü sınıfında **%88 Recall** ve **%87 Precision** ile oldukça yüksek bir başarı yakalanmıştır. Model bu tümör tipini diğerlerinden net bir şekilde ayırabilmektedir.
* **Geliştirilmesi Gereken Alan (Glioma - Meningioma Karışıklığı):** Confusion Matrix incelendiğinde, modelin **Glioma** sınıfındaki örneklerin büyük bir kısmını (151 adet) yanlışlıkla **Meningioma** olarak tahmin ettiği görülmüştür. Bu durum Glioma sınıfının Recall değerinin düşük (%41) kalmasına neden olmuştur.
    * *Sebep:* Tıbbi görüntülemede Glioma ve Meningioma dokuları bazen birbirine çok benzeyebilir. Modelin bu iki sınıfı ayırt etmekte zorlandığı anlaşılmaktadır.

**Sonuç:**
Model, sağlıklı beyinleri ve hipofiz tümörlerini ayırt etmede **uzman seviyesine yakın** performans gösterirken, Glioma ve Meningioma ayrımı için gelecekte daha fazla veri artırma (data augmentation) veya ince ayar (fine-tuning) yapılması gerektiğini göstermektedir. %72'lik genel doğruluk, bu zorlu veri seti için sağlam bir başlangıç noktasıdır.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os


# --- KRİTİK DEĞİŞKENLER  ---
# IMAGE_SIZE: Örn. (224, 224)
# test_path: Test veri setinin ana yolu
# model: Eğitilmiş (veya yeniden yüklenmiş) CNN modeliniz

IMAGE_SIZE = (224,224 )

# Sınıflandırma raporundan aldığınız sınıf isimleri:
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary'] 
LAST_CONV_LAYER_NAME = 'block5_conv3'


# 1. Grad-CAM Isı Haritası Oluşturma Fonksiyonu
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # Gradiente erişmek için bir Keras Model'i oluştur
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        # Görüntüyü gradiente hazırla
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            # En yüksek tahmin edilen sınıfı kullan
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # Son evrişim katmanının çıktılarına göre tahmin edilen sınıfın gradientini al
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # Her kanal için ortalama yoğunluğu al (global average pooling)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Isı haritası oluşturmak için önemli olan kısım: (çıktı * gradient)
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Isı haritasını 0 ile 1 arasına normalleştir
    heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap)
    return heatmap.numpy()

# 2. Isı Haritasını Görüntü Üzerine Çizme Fonksiyonu
# 2. DÜZELTİLMİŞ: Isı Haritasını Görüntü Üzerine Çizme Fonksiyonu
def display_gradcam(img_path, heatmap, pred_index):
    # Orijinal görüntüyü oku
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Isı haritasını 0-255 arasına çek
    heatmap = np.uint8(255 * heatmap)

    # --- KRİTİK EKLEME BURASI ---
    # Isı haritasını (14x14), orijinal resmin boyutuna (örn: 512x512) büyütüyoruz
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    # -----------------------------

    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET) # Renk haritası uygula

    # Isı haritasını görüntü üzerine bindir (saydamlık ile)
    superimposed_img = heatmap * 0.4 + img
    superimposed_img = np.clip(superimposed_img, 0, 255).astype("uint8")

    # Görselleştirme
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(img)
    ax[0].set_title("Orijinal Görüntü")
    ax[0].axis('off')

    ax[1].imshow(superimposed_img)
    ax[1].set_title(f"Grad-CAM (Tahmin: {class_names[pred_index]})")
    ax[1].axis('off')
    plt.show()
    
# --- 3. UYGULAMA: Test Setinden Başarısız Bir Örnek Seçme (Meningioma) ---

# Projenizin en zayıf olduğu sınıf: Meningioma. 
# Bu örnek görselin yolu, sizin Kaggle dizininizde olmalıdır.
# Lütfen bu yolu kontrol edin ve gerekirse doğru yolu buraya girin!

test_path = '/kaggle/input/brain-tumor-mri-dataset/Testing/'
TEST_IMAGE_PATH = os.path.join(test_path, 'meningioma', 'Te-me_0148.jpg')

# Görüntüyü yükle ve modelin beklediği boyuta getir (Örn: 150x150)
img = tf.keras.preprocessing.image.load_img(TEST_IMAGE_PATH, target_size=IMAGE_SIZE)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) # Batch boyutu ekle
img_array = img_array / 255.0 # Normalizasyon

# Modelden tahmin al
preds = model_vgg16.predict(img_array)
pred_index = np.argmax(preds[0]) # En yüksek olasılıklı sınıfın indeksi

print(f"Modelin Tahmini: {class_names[pred_index]} (Olasılık: {preds[0][pred_index]:.2f})")

# Grad-CAM ısı haritasını oluştur
heatmap = make_gradcam_heatmap(img_array, model_vgg16, LAST_CONV_LAYER_NAME, pred_index)

# Görselleştir
display_gradcam(TEST_IMAGE_PATH, heatmap, pred_index)

### 4.4 Model Açıklanabilirliği - Grad-CAM Raporu

Modelin tahminlerinin altında yatan kararları görselleştirmek amacıyla **Grad-CAM** (Gradient-weighted Class Activation Mapping) tekniği başarıyla uygulanmıştır.

**Analiz ve Sonuçlar:**

* Elde edilen ısı haritaları (heatmap), modelin tahmin yaparken rastgele piksellere değil, gerçekten beyindeki tümörlü dokuya odaklandığını göstermektedir.

* Özellikle Pituitary (Hipofiz) sınıfındaki örneklerde, modelin tümörün bulunduğu bölgeyi yüksek aktivasyon (kırmızı/sarı alanlar) ile işaretlediği görülmüştür.

* Bu görselleştirme, modelin sadece ezber yapmadığını, görüntüyü anlamsal olarak doğru yorumladığını ve karar mekanizmasının güvenilir olduğunu kanıtlamaktadır.

### 5. Model Mimarisi ve Optimizasyon Stratejisi

Modelin eğitim seti üzerindeki başarısı ile test seti üzerindeki başarısı arasındaki farkı (Overfitting riskini) minimize etmek için şu stratejiler izlenmiştir:

**Yapısal İyileştirmeler:**

* Transfer Öğrenme (Transfer Learning - VGG16): Sıfırdan bir model eğitmek yerine, ImageNet üzerinde eğitilmiş VGG16 mimarisi kullanılarak modelin öğrenme kapasitesi artırılmıştır.

* Katman Dondurma (Freezing): VGG16'nın alt katmanları dondurularak, modelin temel özellikleri (kenar, doku vb.) kaybetmesi önlenmiş ve eğitilebilir parametre sayısı optimize edilmiştir.

* Dropout ve Regularizasyon: Aşırı öğrenmeyi (overfitting) engellemek amacıyla Dense katmanlarında Dropout uygulanarak modelin genelleme yeteneği artırılmaya çalışılmıştır.

**Gelecek Çalışmalar İçin Öneriler:** Model başarısını daha da artırmak için ilerleyen aşamalarda "Fine-Tuning" (ince ayar) yapılarak VGG16'nın son bloklarının da eğitime dahil edilmesi ve veri artırma (Data Augmentation) tekniklerinin çeşitlendirilmesi önerilmektedir.